In [42]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
import keras
from PIL import Image
import matplotlib.pyplot as plt


print("Setup complete")

Setup complete


In [43]:
img_w,img_h = 64, 64
IMG_DIM = (img_w,img_h)
BATCH_SIZE = 32

PATH = "data/"
LABELS = ["Fully Covered", "Not Covered", "Not a Face", "Partially Covered"]

In [44]:
# creating our training dataset
train_ds = keras.preprocessing.image_dataset_from_directory(
    directory=PATH,
    image_size=(img_h,img_w),
    subset="training",
    seed=123,
    validation_split=0.2)


Found 6621 files belonging to 4 classes.
Using 5297 files for training.


In [45]:
# creating our validation training set
val_ds = keras.preprocessing.image_dataset_from_directory(
    directory=PATH,
    image_size=IMG_DIM,
    subset="validation",
    seed=234,
    validation_split=0.2)


Found 6621 files belonging to 4 classes.
Using 1324 files for validation.


In [46]:
class_names = train_ds.class_names
print(class_names)

['fully_covered', 'not_covered', 'not_face', 'partially_covered']


In [92]:
num_classes = len(class_names)


class Model(keras.Model):

    def __init__(self):
        super().__init__()
        self.input_layer = keras.layers.InputLayer(input_shape=(64, 64, 3))
        self.rescaling_layer = keras.layers.experimental.preprocessing.Rescaling(
            1. / 255, input_shape=(img_h, img_w, 3))
        self.layer_1 = keras.layers.Dense(64, activation='relu')
        self.layer_2 = keras.layers.Dense(32, activation='relu')
        self.layer_3 = keras.layers.Dense(16, activation='relu')
        self.flatten_layer = keras.layers.Flatten()
        self.output_layer = keras.layers.Dense(num_classes, activation='relu')

    def call(self, ds):
        ds = self.input_layer(ds)
        ds = self.flatten_layer(ds)
        ds = self.rescaling_layer(ds)
        ds = self.layer_1(ds)
        ds = self.layer_2(ds)
        ds = self.layer_3(ds)
        ds = self.flatten_layer(ds)
        return self.output_layer(ds)


In [94]:
model = Model()

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

history = model.fit(train_ds, validation_data=val_ds, epochs=5)

model.evaluate(val_ds)


Epoch 1/5
166/166 [==============================] - 1s 8ms/step - loss: 1.3964 - accuracy: 0.2250 - val_loss: 1.3863 - val_accuracy: 0.2168
Epoch 2/5
166/166 [==============================] - 1s 6ms/step - loss: 1.3863 - accuracy: 0.2226 - val_loss: 1.3863 - val_accuracy: 0.2168
Epoch 3/5
166/166 [==============================] - 1s 6ms/step - loss: 1.3863 - accuracy: 0.2226 - val_loss: 1.3863 - val_accuracy: 0.2168
Epoch 4/5
166/166 [==============================] - 1s 7ms/step - loss: 1.3863 - accuracy: 0.2226 - val_loss: 1.3863 - val_accuracy: 0.2168
Epoch 5/5
42/42 [==============================] - 0s 3ms/step - loss: 1.3863 - accuracy: 0.2168


[1.386294960975647, 0.21676737070083618]